In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import rioxarray as riox
import numpy as np
from pathlib import Path
from dask.diagnostics import ProgressBar

In [ ]:
# this image has artifacts
# input_path = Path("/data/psp/test_files/BIO_S2_SCS__1S_20251212T032129_20251212T032150_T_G01_M01_C01_T002_F309_01_DJJGRJ/")
input_path = Path("/data/psp/test_files/BIO_S2_SCS__1S_20251216T034800_20251216T034815_T_G01_M01_C02_T017_F289_01_DJQGAN/")


In [ ]:
# TODO: directly search for abs and phase files

str_tiff = f"**/measurement/*.tiff"
tiffs = list(input_path.glob(str_tiff))
ds = riox.open_rasterio(tiffs[0])
polars = ["hh", "hv", "vh", "vv"]

for it in tiffs:
    if "abs" in str(it):
        amp = riox.open_rasterio(it, chunks={})
    elif "phase" in str(it):
        phi = riox.open_rasterio(it, chunks={})
    else:
        raise ValueError("Unknown tiff file.")

slc = amp * np.exp(1j * phi)

dims = ("y", "x")
S = xr.Dataset(
    {
        "hh": (dims, slc[0].data),
        "hv": (dims, slc[1].data),
        "vh": (dims, slc[2].data),
        "vv": (dims, slc[3].data),
    },
    coords={"y": np.arange(slc.shape[1]), "x": np.arange(slc.shape[2])},
    attrs={"poltype": "S", "description": "Scattering matrix"},
).chunk("auto")

In [ ]:
from polsarpro.util import pauli_rgb, multilook, boxcar, S_to_T3

T3 = S_to_T3(S)
T3 = multilook(T3, 8, 1)
T3 = boxcar(T3, 3, 3)
rgb = pauli_rgb(T3)
rgb.plot.imshow(figsize=(7,7))

In [ ]:
from polsarpro.decompositions import freeman
with ProgressBar():
    out = freeman(T3, boxcar_size=[7, 7]).compute()

In [ ]:
out.double.plot.imshow(vmin=0, vmax=0.1)

In [ ]:
out.odd.plot.imshow(vmin=0, vmax=0.5)

In [ ]:
out.volume.plot.imshow(vmin=0, vmax=0.5)